In [3]:
print('Final Version')

Final Version


In [4]:
!pip install -qU rapidocr-onnxruntime langchain_community pypdf langchain_text_splitters faiss-cpu nse-live-stocks langchain_huggingface langchain_groq langchain_community pypdf2 requests beautifulsoup4 langchain_text_splitters

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-unstructured 0.1.6 requires langchain-core<0.4.0,>=0.3.6, but you have langchain-core 1.0.6 which is incompatible.


In [5]:
import os
import re
import requests
from langchain_groq import ChatGroq
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import FakeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, PDFMinerLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import datetime

/home/pratham/Desktop/projects/virtualenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from sentence_transformers import SentenceTransformer

embeedings = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
sentences = [
    "Hugging Face makes NLP easy.",
    "Transformers are powerful for deep learning."
]
en = embeedings.encode(sentences)
print("Embedding shape:", en.shape)

Embedding shape: (2, 384)


In [7]:
DOWNLOAD_DIR = "pdf_downloads"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

In [8]:
def scrape_screener_pdfs(company_url):
    print(f"Scraping: {company_url}")
    headers = {"User-Agent": "Mozilla/5.0"}

    html = requests.get(company_url, headers=headers).text
    soup = BeautifulSoup(html, "html.parser")

    links = soup.select(".documents a")

    pdf_links = []

    for a in links:
        href = a.get("href", "")
        if href.endswith(".pdf"):
            text = a.text.strip().replace("\n", "_").replace(" ", "_")
            if not text:
                text = "Document"
            pdf_links.append((href, text + ".pdf"))

    print(f"Found {len(pdf_links)} PDF links.\n")
    return pdf_links

In [9]:
def classify_transcript_or_ppt(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = reader.pages[0].extract_text()[:800].lower()
    except:
        return None

    if "transcript" in text or "earnings call" in text:
        return "transcript"
    if "presentation" in text or "investor presentation" in text:
        return "presentation"

    return None


In [10]:
def maybe_rename_transcript_or_ppt(saved_path):
    doc_type = classify_transcript_or_ppt(saved_path)

    if not doc_type:
        print("   → Not transcript/presentation. Keeping original filename.\n")
        return saved_path

    # Original filename
    folder = os.path.dirname(saved_path)
    base = os.path.basename(saved_path)
    name, ext = os.path.splitext(base)

    # Make new filename with incremental number
    counter = 1
    new_name = f"{name}_{counter}{ext}"
    new_path = os.path.join(folder, new_name)

    while os.path.exists(new_path):
        counter += 1
        new_name = f"{name}_{counter}{ext}"
        new_path = os.path.join(folder, new_name)

    try:
        os.rename(saved_path, new_path)
        print(f"   ✔ Transcript/PPT detected → renamed to {new_name}\n")
        return new_path
    except:
        print("   ⚠ Rename failed. Keeping original.\n")
        return saved_path


In [11]:
def download_bse_annpdf(url, download_dir, filename):
    print(f"URL: {url}")
    print(f"[BSE-ANNPDF] Requesting: {url}")

    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    savepath = os.path.join(download_dir, filename)

    with open(savepath, "wb") as f:
        f.write(r.content)

    print(f"✔ Saved BSE AnnPdf: {savepath}")
    return savepath

In [12]:
def download_bse_iframe_pdf(url, download_dir, filename):
    print(f"URL: {url}")
    print("   [BSE-IFRAME] Requesting main page…")

    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}).text
    soup = BeautifulSoup(html, "html.parser")
    iframe = soup.find("iframe")

    if not iframe:
        print("   ❌ No iframe found. Cannot download.")
        return None

    real_pdf = iframe.get("src")
    if not real_pdf.startswith("http"):
        real_pdf = "https://www.bseindia.com" + real_pdf

    print(f"   → PDF Source: {real_pdf}")

    r = requests.get(real_pdf, headers={"User-Agent": "Mozilla/5.0"})
    savepath = os.path.join(download_dir, filename)

    with open(savepath, "wb") as f:
        f.write(r.content)

    print(f"   ✔ Saved BSE iframe PDF: {savepath}")
    return savepath


In [13]:
def download_direct_pdf(url, download_dir, filename):
    print(f"[DIRECT] Downloading: {url}")
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    savepath = os.path.join(download_dir, filename)

    with open(savepath, "wb") as f:
        f.write(r.content)

    print(f"✔ Saved direct PDF: {savepath}")
    return savepath

In [14]:
def clean_filename(name):
    name = re.sub(r'[\/:*?"<>|;,]', '_', name)
    name = re.sub(r"_+", "_", name)
    name = name.strip(" .")
    if len(name) > 180:
        name = name[:180]
    return name

In [15]:
def download_pdf(url, filename):

    filename = clean_filename(filename)

    if "AnnPdfOpen.aspx" in url:
        saved = download_bse_annpdf(url, DOWNLOAD_DIR, filename)
        return maybe_rename_transcript_or_ppt(saved)

    if "xml-data/corpfiling" in url:
        saved = download_bse_iframe_pdf(url, DOWNLOAD_DIR, filename)
        return maybe_rename_transcript_or_ppt(saved)

    saved = download_direct_pdf(url, DOWNLOAD_DIR, filename)
    return maybe_rename_transcript_or_ppt(saved)


In [16]:
def run(company_url):
    pdfs = scrape_screener_pdfs(company_url)

    for url, filename in pdfs:
        print("-----------------------------------------")
        print(f"Downloading: {filename}")
        # download_pdf(url, filename)


In [17]:
url = 'https://www.screener.in/company/JIOFIN/consolidated/'

run(url)

Scraping: https://www.screener.in/company/JIOFIN/consolidated/
Found 26 PDF links.

-----------------------------------------
Downloading: Update_On_Institutional_Investors''_Meeting_________________________1d_-_Nov_17,_2025_London_in-person_non-deal_roadshow_with_Investec;_no_unpublished_price-sensitive_information..pdf
-----------------------------------------
Downloading: Announcement_under_Regulation_30_(LODR)-Press_Release_/_Media_Release_________________________2d_-_Nov_17,_2025:_JioFinance_app_launches_unified_dashboard_to_link_and_analyse_accounts,_mutual_funds,_equities..pdf
-----------------------------------------
Downloading: Announcement_under_Regulation_30_(LODR)-Acquisition_________________________14_Nov_-_JLSL_subscribed_4,50,00,000_8.1%_OCPS_for_Rs45_crore_in_RILIPL;_aggregate_investment_Rs166.55_crore..pdf
-----------------------------------------
Downloading: Announcement_under_Regulation_30_(LODR)-Analyst_/_Investor_Meet_-_Intimation_________________________11_Nov_-

In [ ]:
def delete_old_pdfs():

    FOLDER = "pdf_downloads/"
    ONE_YEAR_DAYS = 365

    def parse_pdf_date(date_str):
        # Format type 1: D:20181012055359+05'30'
        if date_str.startswith("D:"):
            try:
                dt = datetime.datetime.strptime(date_str[2:16], "%Y%m%d%H%M%S")
                return dt
            except:
                pass

        # Format type 2: Fri 12 Oct 2018 05:53:59 PM +05:30
        try:
            dt = datetime.datetime.strptime(date_str, "%a %d %b %Y %I:%M:%S %p %z")
            return dt.replace(tzinfo=None)
        except:
            return None


    now = datetime.datetime.now()
    deleted_files = 0

    for filename in os.listdir(FOLDER):
        if not filename.lower().endswith(".pdf"):
            continue
        
        file_path = os.path.join(FOLDER, filename)

        try:
            reader = PdfReader(file_path)
            metadata = reader.metadata
            
            if "/CreationDate" in metadata:
                pdf_date = parse_pdf_date(metadata["/CreationDate"])
            elif "/ModDate" in metadata:
                pdf_date = parse_pdf_date(metadata["/ModDate"])
            else:
                print(f"⚠ No metadata date found for {filename}, skipping.")
                continue

            if not pdf_date:
                print(f"⚠ Could not parse date for {filename}, skipping.")
                continue

            age_days = (now - pdf_date).days

            if age_days > ONE_YEAR_DAYS:
                print(f"🗑 Deleting: {filename} (Age: {age_days} days)")
                os.remove(file_path)
                deleted_files += 1
            else:
                print(f"✔ Keeping: {filename} (Age: {age_days} days)")

        except Exception as e:
            print(f"❌ Error reading {filename}: {e}")

    print(f"\n✅ Done. Deleted {deleted_files} old PDF(s).")

    return f'Deleted {deleted_files} old PDF(s).'

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# note: UnstructuredLoader is already available in the notebook environment
from langchain_unstructured import UnstructuredLoader

# reuse existing global `vector_store` if present
vector_store = globals().get("vector_store", None)
count = 0
file_path = []

# collect full paths to PDFs in DOWNLOAD_DIR
for fname in os.listdir(DOWNLOAD_DIR):
    if fname.lower().endswith(".pdf"):
        file_path.append(os.path.join(DOWNLOAD_DIR, fname))
    
print(f'Length of Files in Folder is {len(file_path)}')

try:
    if not file_path:
        raise ValueError(f"No PDF files found in {DOWNLOAD_DIR}")

    loader = UnstructuredLoader(file_path)
    docs = loader.load()

    chunks = text_splitter.split_documents(docs)
    print(f'Total length of chunks stored into db is {len(chunks)}')

    # create a new FAISS store if none exists, otherwise add documents

except Exception as e:
    print(f'Error processing files: {e}')




Length of Files in Folder is 1


INFO: pikepdf C++ to Python logger bridge initialized


Total length of chunks stored into db is 10654
Error processing files: name 'embeddings' is not defined


In [26]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False}
)


INFO: Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


In [28]:
if vector_store is None:
    vector_store = FAISS.from_documents(chunks, embeddings_model)
    print("Created new FAISS vector store from chunks.")
else:
    vector_store.add_documents(chunks)
    print("Added chunks to existing vector store.")


INFO: Loading faiss with AVX512 support.
INFO: Successfully loaded faiss with AVX512 support.


Created new FAISS vector store from chunks.


In [29]:
from langchain_community.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(urls=[url])
data = loader.load()
chunks = text_splitter.split_documents(data)
vector_store=FAISS.from_documents(chunks,embeddings_model)

In [ ]:
vector_store

In [30]:
query = 'what is last 2 year profit anlaysis'

results = vector_store.similarity_search(query)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* Jun 2023 Sep 2023 Dec 2023 Mar 2024 Jun 2024 Sep 2024 Dec 2024 Mar 2025 Jun 2025 Sep 2025 Sales + 414 608 414 418 418 694 438 493 612 981 Expenses + 38 66 94 98 74 140 125 155 156 293 Operating Profit 376 542 320 320 344 553 313 338 457 688 OPM % 91% 89% 77% 77% 82% 80% 71% 69% 75% 70% Other Income + 67 218 67 78 62 226 70 71 67 238 Interest 10 0 0 0 0 0 0 8 99 136 Depreciation 5 5 5 5 5 6 6 6 6 8 Profit before tax 427 754 381 393 400 773 377 396 419 783 Tax % 22% 11% 23% 21% 22% 11% 22% 20% [{'source': 'https://www.screener.in/company/JIOFIN/consolidated/'}]
* 396 419 783 Tax % 22% 11% 23% 21% 22% 11% 22% 20% 23% 11% Net Profit + 332 668 294 311 313 689 295 316 325 695 EPS in Rs 1.05 0.46 0.49 0.49 1.08 0.46 0.50 0.51 1.09 Raw PDF [{'source': 'https://www.screener.in/company/JIOFIN/consolidated/'}]
* Profit & Loss

Consolidated Figures in Rs. Crores / View Standalone

Mar 2023 Mar 2024 Mar 2025 TTM Sales + 45 1,855 2,043 2,525 Expenses + 6 296 495 729 Operating Profit 39 1,559 1,549

In [34]:
query = 'what is last 2 year profit anlaysis'

result = vector_store.similarity_search_with_score(query,k=3)

result

[(Document(id='51caec11-2a3a-4bea-aa3d-98c5e8c247b0', metadata={'source': 'https://www.screener.in/company/JIOFIN/consolidated/'}, page_content='Jun 2023 Sep 2023 Dec 2023 Mar 2024 Jun 2024 Sep 2024 Dec 2024 Mar 2025 Jun 2025 Sep 2025 Sales + 414 608 414 418 418 694 438 493 612 981 Expenses + 38 66 94 98 74 140 125 155 156 293 Operating Profit 376 542 320 320 344 553 313 338 457 688 OPM % 91% 89% 77% 77% 82% 80% 71% 69% 75% 70% Other Income + 67 218 67 78 62 226 70 71 67 238 Interest 10 0 0 0 0 0 0 8 99 136 Depreciation 5 5 5 5 5 6 6 6 6 8 Profit before tax 427 754 381 393 400 773 377 396 419 783 Tax % 22% 11% 23% 21% 22% 11% 22% 20%'),
  np.float32(0.7555691)),
 (Document(id='d56ed291-7ada-4176-86f0-afd58c6b6a2f', metadata={'source': 'https://www.screener.in/company/JIOFIN/consolidated/'}, page_content='396 419 783 Tax % 22% 11% 23% 21% 22% 11% 22% 20% 23% 11% Net Profit + 332 668 294 311 313 689 295 316 325 695 EPS in Rs 1.05 0.46 0.49 0.49 1.08 0.46 0.50 0.51 1.09 Raw PDF'),
  np.fl

In [ ]:
def create_url_vector_store(url):

    url_loader = WebBaseLoader(url,embeedings)

    url_docs = url_loader.load()

    url_split_documents = text_splitter.split_documents(url_docs)

    vector_store.add_documents(url_split_documents)

    vector_store.save_local("faiss_index")

    vector_store = vector_store.load_local("faiss_index", 
                                           embeddings=embeedings,
                                           allow_dangerous_deserialization=True)
    
    return vector_store

In [36]:
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY=os.environ['GROQ_API_KEY']

In [ ]:
vector_store = create_vector_store(vector_store,embeedings)
vector_store = create_url_vector_store(url,embeedings)

In [ ]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    api_key=''
    # other params...
)

llm.invoke('tell me one liner joke on OLA share')

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Why did the OLA share go to therapy? Because it kept getting “driven” crazy by market traffic!', additional_kwargs={'reasoning_content': "User wants a one-liner joke about OLA share. Likely about Ola (ride-hailing) or OLA (stock ticker). Provide a joke. Ensure it's appropriate."}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 80, 'total_tokens': 149, 'completion_time': 0.141381706, 'completion_tokens_details': {'reasoning_tokens': 37}, 'prompt_time': 0.003072989, 'prompt_tokens_details': None, 'queue_time': 0.04785852, 'total_time': 0.144454695}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_1d1727abc9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--de2bc8c8-fe07-4893-bc3e-c12f51c1a057-0', usage_metadata={'input_tokens': 80, 'output_tokens': 69, 'total_tokens': 149, 'output_token_details': {'reasoning': 37}})

In [42]:
base_prompt = '''
You are an intelligent financial analysis agent specialized in reviewing company quarterly reports and earnings call transcripts.
Your primary function is to extract accurate financial metrics, analyze qualitative management commentary, and generate clear, structured insights.

During execution, you will receive:

{context}: Extracted text chunks from financial reports and transcripts

{question}: A specific analytical task (e.g., trends, risks, sentiment, outlook)

Guidelines for responding to {question} using {context}:

Accurate & Grounded: Use only the information found in the provided context—no guessing or fabricating data.

Financially Insightful: Provide concise explanations about revenue trends, margin movements, demand commentary, risks, and opportunities.

Forecast-Oriented: Highlight management sentiment and forward-looking statements relevant to future performance.

Structured JSON: Always respond in a predictable JSON format suitable for downstream processing.

No Investment Advice: Do not provide stock buy/sell/hold recommendations or personal financial advice.

Example JSON Output:
{{
  "reply": "Revenue grew 5% YoY driven by cloud and BFSI demand, while margins remained stable. Management highlighted healthy deal wins and improving client budgets.",
  "guidance_caution": "This summary is based solely on the provided financial context and does not constitute investment advice.",
  "follow_up_prompt": "Would you like insights on risks, opportunities, or the outlook for the next quarter?"
}}

'''

In [47]:
user_question_2 = 'What were the key financial metrics (Revenue, Net Profit, Operating Margin, and Segment Performance) reported for this quarter?'

prompt = PromptTemplate(template=base_prompt, input_variables=["context", "question"])
vectorstore_retreiver = vector_store.as_retriever(search_kwargs={"k": 3})
extracted_chunks = vectorstore_retreiver.invoke(user_question_2)
chain = prompt | llm | JsonOutputParser()
response = chain.invoke({"context": extracted_chunks, "question": user_question_2})

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [48]:
def user_query_answer(query):
    vectorstore_retreiver = vector_store.as_retriever(search_kwargs={"k": 3})
    extracted_chunks = vectorstore_retreiver.invoke(query)
    chain = prompt | llm | JsonOutputParser()
    response = chain.invoke({"context": extracted_chunks, "question": query})
    return response, extracted_chunks

In [49]:
extracted_chunks

[Document(id='51caec11-2a3a-4bea-aa3d-98c5e8c247b0', metadata={'source': 'https://www.screener.in/company/JIOFIN/consolidated/'}, page_content='Jun 2023 Sep 2023 Dec 2023 Mar 2024 Jun 2024 Sep 2024 Dec 2024 Mar 2025 Jun 2025 Sep 2025 Sales + 414 608 414 418 418 694 438 493 612 981 Expenses + 38 66 94 98 74 140 125 155 156 293 Operating Profit 376 542 320 320 344 553 313 338 457 688 OPM % 91% 89% 77% 77% 82% 80% 71% 69% 75% 70% Other Income + 67 218 67 78 62 226 70 71 67 238 Interest 10 0 0 0 0 0 0 8 99 136 Depreciation 5 5 5 5 5 6 6 6 6 8 Profit before tax 427 754 381 393 400 773 377 396 419 783 Tax % 22% 11% 23% 21% 22% 11% 22% 20%'),
 Document(id='90530832-ad8b-48de-85b3-aa9390366bde', metadata={'source': 'https://www.screener.in/company/JIOFIN/consolidated/'}, page_content='Profit & Loss\n\nConsolidated Figures in Rs. Crores / View Standalone\n\nMar 2023 Mar 2024 Mar 2025 TTM Sales + 45 1,855 2,043 2,525 Expenses + 6 296 495 729 Operating Profit 39 1,559 1,549 1,797 OPM % 88% 84% 76

In [51]:
print(response)

{'reply': 'The most recent quarterly data (September\u202f2025) shows:\n- **Revenue (Sales)**:\u202f₹981\u202fcrore\n- **Operating Profit**:\u202f₹688\u202fcrore\n- **Operating Margin (OPM)**:\u202f70\u202f%\n- **Other Income**:\u202f₹238\u202fcrore (including interest\u202f₹136\u202fcrore and depreciation\u202f₹8\u202fcrore)\n- **Profit before Tax**:\u202f₹783\u202fcrore\n- **Tax Rate**:\u202f20\u202f%\n- **Net Profit**: Not disclosed directly; applying the reported tax rate would imply a net profit of roughly ₹626\u202fcrore (₹783\u202f×\u202f(1‑0.20)).\n\n**Segment performance**: No segment‑level revenue or profit figures are provided in the supplied excerpts, so a breakdown by business line cannot be reported.\n\n**Key observations**:\n- Revenue has risen sharply from ₹414\u202fcrore in Dec\u202f2023 to ₹981\u202fcrore in Sep\u202f2025, indicating strong top‑line growth.\n- Operating margin remains high at 70\u202f%, though it has slipped from a peak of 91\u202f% in Jun\u202f2023, 

In [ ]:
from nse_live_stocks import Nse

stock = Nse()

result = stock.get_current_price('TCS')

print(result)